In [3]:
%pip install numpy pandas scipy matplotlib pingouin openpyxl


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

dataframe: pd.DataFrame = pd.read_excel("../analysis/analysis.xlsx")
dataframe.head()

,experiment,model,version,date,metric,apathetic_score,joy_score,fear_score,anger_score
0,BooleanQAExperiment,gemma2,9b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.882,0.874,0.872,0.866
1,BooleanQAExperiment,llama3.1,8b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.790,0.794,0.696,0.816
2,BooleanQAExperiment,llama3.2,3b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.478,0.686,0.640,0.670
3,BooleanQAExperiment,mistral,7b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.604,0.684,0.724,0.656
4,BooleanQAExperiment,qwen2.5,7b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.810,0.784,0.730,0.784


In [6]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from itertools import combinations
import pingouin as pg

# Supponiamo di avere accuracy per un task e ogni emozione
data = {
    "Apatia": dataframe["apathetic_score"].to_list(),
    "Gioia": dataframe["joy_score"].to_list(),
    "Paura": dataframe["fear_score"].to_list(),
    "Rabbia": dataframe["anger_score"].to_list()
}

def cohen_d(group1, group2):
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    pooled_std = np.sqrt((std1**2 + std2**2) / 2)
    return (mean1 - mean2) / pooled_std

# Calcolo di Cohen's d per ogni coppia di emozioni
cohen_results = {}
for emo1, emo2 in combinations(data.keys(), 2):
    cohen_results[f"{emo1} vs {emo2}"] = cohen_d(data[emo1], data[emo2])

# Creazione DataFrame per ANOVA
df = pd.DataFrame([(score, emotion) for emotion, scores in data.items() for score in scores], columns=["accuracy", "emotion"])

# ANOVA con Eta-squared
anova = pg.anova(dv="accuracy", between="emotion", data=df, detailed=True)
eta_squared = anova.loc[0, "np2"]  # np2 = eta-squared

cohen_results, eta_squared
#cohen_results

({'Apatia vs Gioia': -0.0637293636295744,
  'Apatia vs Paura': 0.10531281357597377,
  'Apatia vs Rabbia': 0.014596100915157196,
  'Gioia vs Paura': 0.17717781438816935,
  'Gioia vs Rabbia': 0.08030141752366597,
  'Paura vs Rabbia': -0.09200092715049064},
 0.003847660411084469)